In [1]:
import numpy as np

In [2]:
###
### Nothing loads temporarily. Later it should be overload.
###

def load_parameters():
    X = np.random.randn(64*64, 200)
    Y = np.random.randn(2, 200)
    return X, Y

In [3]:
def initialize_parameters(X, hidden_layers, Y, sigma):
    W, b = {}, {}
    n_L = len(hidden_layers)      # number of layers

    # Init (n_H, n_W) matrix for each layer exclude output
    n_W = X.shape[0]              # width
    for i in range(n_L):
        n_H = hidden_layers[i]    # height
        l = str(i + 1)            # index of layer
        W["W" + l] = np.random.randn(n_H, n_W) * sigma
        b["b" + l] = np.zeros((n_H, 1))
        n_W = n_H                 # the next layer contains the same width as the height of the previous layer

    # Init (n_H, n_W) matrix for output
    l = str(n_L + 1)
    n_H = Y.shape[0]              # final height, number of outputs
    W["W" + l] = np.random.randn(n_H, n_W) * sigma
    b["b" + l] = np.zeros((n_H, 1))

    return W, b

In [4]:
def relu(Z):
    A = Z * (Z > 0)
    return A

In [5]:
def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    return A

In [6]:
def single_step_propagation(A_prev, W, b, activation):
    Z = np.dot(W, A_prev) + b
    
    if activation == "relu":
        A = relu(Z)

    elif activation == "sigmoid":
        A = sigmoid(Z)

    return A

In [7]:
def cost_calculation(A, Y):
    m = Y.shape[1]
    return -np.sum((Y * np.log(A) + (1 - Y) * np.log(1 - A))) / m

In [8]:
def forward_propagation(X, W, b, Y):
    A = {}                        # A[i] = g(W[i] * X[i-1] + b[i]), it's need for backward propagation too 
    n_L = len(W)                  # number of layers

    # each layer uses ReLU for activation
    A_prev = X
    for l in range(1, n_L):
        A["A" + str(l)] = single_step_propagation(A_prev, W["W" + str(l)], b["b" + str(l)], "relu")
        A_prev = A["A" + str(l)]

    # last layer uses Sigmoid for activation
    A["A" + str(n_L)] = single_step_propagation(A_prev, W["W" + str(n_L)], b["b" + str(n_L)], "sigmoid")

    cost = cost_calculation(A["A" + str(n_L)], Y)

    return cost, A

In [9]:
def backward_sigmoid(W, b, A, dW, db):
    return dW, db

In [10]:
def backward_relu(W, b, A, dW, db):
    return dW, db

In [11]:
def backward_propogation(W, b, A):
    dW, db = {}, {}
    dW, db = backward_sigmoid(W, b, A, dW, db)
    dW, db = backward_relu(W, b, A, dW, db)
    return dW, db

In [12]:
def model(X, Y, hidden_layers, learning_rate=0.01, sigma=0.01, iteration=1000):
    W, b = initialize_parameters(X, hidden_layers, Y, sigma)
    for i in range(iteration):
        cost, A = forward_propagation(X, W, b, Y)
        dW, db = backward_propogation(W, b, A)
        if i % 100 == 0:
            print("cost =", cost)